In [1]:
!pip install elasticsearch[async]==7.17.0 -q
!pip install elasticsearch-dsl==7.4.0 -q

In [1]:
import sys

sys.path.append('/app')

In [2]:
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd
import ast
import json

In [3]:
from memlish.config import ESTag

In [4]:
client = Elasticsearch("http://35.224.116.253:7032")
index = "fluentd-*"

In [5]:
# client.delete_by_query(index=[index], body={"query": {"match_all": {}}}, size=1000000)

In [6]:
# client.delete_by_query(index=[index], body={"query": {
#     "bool": {
#       "must": {
#         "match": {
#           "log": ESTag.TIMELOG
#         }
#       }
#     }
#   }}, size=1000000)

In [7]:
# time_logs = []
# time_log_search = Search(using=client, index=index).query("multi_match", query=ESTag.TIMELOG, fields=['log'])

# for hit in time_log_search.scan():
#     _hit = {}
#     _hit = ast.literal_eval(hit.to_dict()['log'])
#     time_logs.append(_hit)

In [8]:
client.info()

{'name': 'elastic',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'QeWlxU4zT4STOl7p7e6P9g',
 'version': {'number': '7.0.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'e4efcb5',
  'build_date': '2019-04-29T12:56:03.145736Z',
  'build_snapshot': False,
  'lucene_version': '8.0.0',
  'minimum_wire_compatibility_version': '6.7.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [9]:
answ_search = Search(using=client, index=index).query("multi_match", query=ESTag.INLINE_ANSWER, fields=['log'])
choice_search = Search(using=client, index=index).query("multi_match", query=ESTag.INLINE_CHOICE, fields=['log'])

In [20]:
#cand_id - is guid. Example: 3940172d-fda5-4e02-b284-6f5e2074454c 
def find_choice_for_answer(cand_id, choices):
    for i in choices:
        choice_id = i['update']['result_id'].split('_')[1]
        if choice_id == cand_id:
            return i

In [24]:
choice_logs = []
rows = []

for hit in choice_search.scan():
    _hit = {}
    _hit = ast.literal_eval(hit.to_dict()['log'])
    _hit['update'] = json.loads(_hit['update'])
    _hit["timestamp"] = hit["@timestamp"]
    choice_logs.append(_hit)

for hit in answ_search.scan():
    result_row = {}
    _hit = ast.literal_eval(hit.to_dict()['log'])
    _hit["answer_timestamp"] = hit["@timestamp"]
    if len(_hit['template_options']) > 0:
        _hit_template_uniq_id = _hit['template_options'][0]['id'].split('_')[1]
        possible_choice = find_choice_for_answer(_hit_template_uniq_id, choice_logs)
    
    update = json.loads(_hit['update'])
    
    if possible_choice is not None:
        choice_update = possible_choice['update']
        result_row["template_choice"] = choice_update["result_id"]
        result_row["choice_timestamp"] = possible_choice['timestamp']
    
    result_row["answer_timestamp"] = _hit["answer_timestamp"]
    result_row["id"] = update["id"]
    result_row["user_id"] = update["from"]["id"]
    result_row["first_name"] = update["from"]["first_name"]
    result_row["last_name"] = update["from"].get("last_name", "")
    result_row["username"] = update["from"]["username"]
    result_row["language_code"] = update["from"]["language_code"]
    result_row["chat_type"] = update["chat_type"]
    result_row["query"] = update["query"]
    result_row["candidates"] = _hit["template_options"]
    
    rows.append(result_row)

In [25]:
logs_df = pd.DataFrame(rows)

In [32]:
banned_users = ['daniel_kovalenko', 'ParadiseN', 'kilianovski', 'ostap_hembara', 'CheshireDog1865']

banned_mask = logs_df['username'].apply(lambda x: x not in banned_users)

In [33]:
logs_df[banned_mask]['username'].value_counts()

Series([], Name: username, dtype: int64)